# José Ricardo Jáuregui Guevara - 608995

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

El conjunto de datos consta de 395 filas y 10 variables, que incluyen información demográfica (Escuela, Sexo, Edad), hábitos de estudio (HorasDeEstudio, Internet), historial académico (Reprobadas, Faltas) y calificaciones (G1, G2, G3). La variable objetivo es G3, que representa la calificación final del curso. Identificamos tres variables categóricas (Escuela, Sexo, Internet) que requieren transformación para ser utilizadas en un modelo de regresión lineal, ya que este tipo de modelos solo acepta variables numéricas. Las variables restantes son numéricas, lo que facilita su inclusión directa en el análisis. Esta exploración inicial permite comprender la estructura de los datos y anticipar los retos de modelado, como la codificación de categorías y la posible existencia de relaciones complejas entre variables.

In [97]:
df = pd.read_csv('calif.csv')
print(df.shape)
df.head()

(395, 10)


,Escuela,Sexo,Edad,HorasDeEstudio,Reprobadas,Internet,Faltas,G1,G2,G3
0,GP,F,18,2,0,no,6,5,6,6
1,GP,F,17,2,0,yes,4,5,5,6
2,GP,F,15,2,3,yes,10,7,8,10
3,GP,F,15,3,0,yes,2,15,14,15
4,GP,F,16,2,0,no,4,6,10,10


In [98]:
df.dtypes

Escuela           object
Sexo              object
Edad               int64
HorasDeEstudio     int64
Reprobadas         int64
Internet          object
Faltas             int64
G1                 int64
G2                 int64
G3                 int64
dtype: object

In [99]:
df['Sexo'].value_counts()

Sexo
F    208
M    187
Name: count, dtype: int64

Para garantizar la compatibilidad con los algoritmos de regresión lineal, se transformaron todas las variables categóricas a representaciones numéricas mediante la técnica de Label Encoding. La variable "Sexo" se codificó como 0 (Femenino) y 1 (Masculino); "Escuela" como 0 (GP) y 1 (MS); e "Internet" como 0 (no) y 1 (yes). Este proceso es necesario porque los modelos lineales no pueden interpretar directamente etiquetas textuales. No se detectaron valores faltantes en el conjunto, por lo que no fue necesario realizar imputación. Tampoco se identificaron valores atípicos extremos que requirieran eliminación.

In [100]:
# Convertimos las variables categóricas a categóricas númericas
from sklearn.preprocessing import LabelEncoder

# Aplicamos la transformación
df['Sexo_num'] = LabelEncoder().fit_transform(df['Sexo'])
df.sample(5)
# Sexo - 0 = F
# Sexo - 1 = M

,Escuela,Sexo,Edad,HorasDeEstudio,Reprobadas,Internet,Faltas,G1,G2,G3,Sexo_num
292,GP,F,18,2,1,yes,12,12,12,13,0
365,MS,M,18,2,0,no,4,10,10,10,1
280,GP,M,17,1,0,yes,30,8,8,8,1
279,GP,M,18,1,0,yes,8,10,11,10,1
376,MS,F,20,3,2,yes,4,15,14,15,0


In [101]:
df['Escuela'].value_counts()

Escuela
GP    349
MS     46
Name: count, dtype: int64

In [102]:
# Convertimos las variables categóricas a categóricas númericas
from sklearn.preprocessing import LabelEncoder

# Aplicamos la transformación
df['Escuela_num'] = LabelEncoder().fit_transform(df['Escuela'])
df.sample(5)
# Escuela - 0 = GP
# Escuela - 1 = MS

,Escuela,Sexo,Edad,HorasDeEstudio,Reprobadas,Internet,Faltas,G1,G2,G3,Sexo_num,Escuela_num
334,GP,F,18,4,0,no,0,10,9,0,0,0
169,GP,F,16,2,0,yes,0,14,14,14,0,0
37,GP,M,16,3,0,yes,7,15,16,15,1,0
197,GP,M,16,1,0,yes,8,9,9,10,1,0
88,GP,M,16,2,1,yes,12,11,10,10,1,0


In [103]:
df['Internet'].value_counts()

Internet
yes    329
no      66
Name: count, dtype: int64

In [104]:
# Convertimos las variables categóricas a categóricas númericas
from sklearn.preprocessing import LabelEncoder

# Aplicamos la transformación
df['Internet_num'] = LabelEncoder().fit_transform(df['Internet'])
df.sample(5)
# Internet - 0 = no
# Internet - 1 = yes

,Escuela,Sexo,Edad,HorasDeEstudio,Reprobadas,Internet,Faltas,G1,G2,G3,Sexo_num,Escuela_num,Internet_num
9,GP,M,15,2,0,yes,0,14,15,15,1,0,1
96,GP,M,16,1,0,yes,2,11,15,15,1,0,1
68,GP,F,15,2,0,yes,2,8,9,8,0,0,1
351,MS,M,17,2,0,yes,2,13,13,13,1,1,1
268,GP,M,18,2,0,yes,10,10,9,10,1,0,1


Después de hacer al análisis inicial, se creó una tabla de correlación sobre cada una de las variables en el conjunto de datos. 
La tabla de correlación revela correlaciones importantes entre las variables. Existe una alta correlación entre las calificaciones G1, G2 y G3, lo que indica alguna multicolinealidad si se incluyen todas en el modelo. La variable "Reprobadas" muestra una correlación negativa moderada con G3 (-0.36), indicando que a mayor número de materias reprobadas, menor calificación final. "Edad" tiene una correlación negativa débil con G3 (-0.16), mientras que "HorasDeEstudio" presenta una correlación positiva baja (0.10). 

In [105]:
df.corr(numeric_only=True)

,Edad,HorasDeEstudio,Reprobadas,Faltas,G1,G2,G3,Sexo_num,Escuela_num,Internet_num
Edad,1.000000,-0.004140,0.243665,0.175230,-0.064081,-0.143474,-0.161579,-0.028606,0.377610,-0.112094
HorasDeEstudio,-0.004140,1.000000,-0.173563,-0.062700,0.160612,0.135880,0.097820,-0.306268,-0.090681,0.059422
Reprobadas,0.243665,-0.173563,1.000000,0.063726,-0.354718,-0.355896,-0.360415,0.044436,0.059804,-0.063451
Faltas,0.175230,-0.062700,0.063726,1.000000,-0.031003,-0.031777,0.034247,-0.066962,-0.088480,0.101701
G1,-0.064081,0.160612,-0.354718,-0.031003,1.000000,0.852118,0.801468,0.091839,-0.025731,0.071619
G2,-0.143474,0.135880,-0.355896,-0.031777,0.852118,1.000000,0.904868,0.091099,-0.050086,0.119439
G3,-0.161579,0.097820,-0.360415,0.034247,0.801468,0.904868,1.000000,0.103456,-0.045017,0.098483
Sexo_num,-0.028606,-0.306268,0.044436,-0.066962,0.091839,0.091099,0.103456,1.000000,-0.012286,0.044113
Escuela_num,0.377610,-0.090681,0.059804,-0.088480,-0.025731,-0.050086,-0.045017,-0.012286,1.000000,-0.133578
Internet_num,-0.112094,0.059422,-0.063451,0.101701,0.071619,0.119439,0.098483,0.044113,-0.133578,1.000000


In [106]:
model_FS = smf.ols(formula='G3~Edad+HorasDeEstudio+Reprobadas+Faltas+G1+G2+Sexo_num+Escuela_num+Internet_num', data=df).fit()
model_FS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     G3   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     211.2
Date:                Thu, 05 Feb 2026   Prob (F-statistic):          5.55e-143
Time:                        08:52:48   Log-Likelihood:                -809.36
No. Observations:                 395   AIC:                             1639.
Df Residuals:                     385   BIC:                             1679.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.7950      1.481      1.212      0.226      -1.117       4.707
Edad              -0.2020      0.087     -2.321      0.021      -0.373      -0.031
HorasDeEstudio    -0.1109      0.125     -0.888      0.375      -0.356       0.134
Reprobadas        -0.2203      0.144     -1.531      0.126      -0.503       0.063
Faltas             0.0453      0.013      3.618      0.000       0.021       0.070
G1                 0.1604      0.056      2.847      0.005       0.050       0.271
G2                 0.9643      0.050     19.372      0.000       0.866       1.062
Sexo_num           0.1905      0.205      0.927      0.354      -0.213       0.594
Escuela_num        0.3436      0.331      1.039      0.299      -0.307       0.994
Internet_num      -0.2149      0.264     -0.815      0.415      -0.733       0.303
==============================================================================
Omnibus:                      206.998   Durbin-Watson:                   1.897
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1137.089
Skew:                          -2.264   Prob(JB):                    1.21e-247
Kurtosis:                       9.971   Cond. No.                         370.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Mediante un proceso de statsmodels, se evaluó la significancia de cada variable. El modelo inicial incluyó todas las variables pero muchas presentaron p-valores superiores a 0.05, lo que indica baja significancia. Tras varios ajustes se seleccionaron cuatro variables: G2 (calificación del segundo período), Reprobadas (número de materias reprobadas), Faltas (inasistencias) y Sexo_num (género codificado). G2 es el predictor más fuerte (p=0), Reprobadas y Faltas tienen p-valores inferiores a 0.05, y Sexo_num se incluyó para ver posibles diferencias de género. Variables como Edad, HorasDeEstudio, Escuela_num e Internet_num se descartaron por no aportar significancia.

In [107]:
model_FS = smf.ols(formula='G3 ~ G2 + Reprobadas + Sexo_num + Faltas', data=df).fit()
model_FS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     G3   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     461.6
Date:                Thu, 05 Feb 2026   Prob (F-statistic):          2.03e-146
Time:                        08:52:48   Log-Likelihood:                -816.25
No. Observations:                 395   AIC:                             1643.
Df Residuals:                     390   BIC:                             1662.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3955      0.342     -4.082      0.000      -2.068      -0.723
G2             1.0795      0.028     38.912      0.000       1.025       1.134
Reprobadas    -0.3117      0.140     -2.224      0.027      -0.587      -0.036
Sexo_num       0.2702      0.196      1.381      0.168      -0.115       0.655
Faltas         0.0387      0.012      3.183      0.002       0.015       0.063
==============================================================================
Omnibus:                      229.939   Durbin-Watson:                   1.871
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1523.912
Skew:                          -2.495   Prob(JB):                         0.00
Kurtosis:                      11.227   Cond. No.                         48.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [108]:
model_FS.pvalues

Intercept      5.423939e-05
G2            2.352502e-136
Reprobadas     2.671466e-02
Sexo_num       1.681952e-01
Faltas         1.574704e-03
dtype: float64

In [109]:
pvalue_percent = (model_FS.pvalues*100).sort_values(ascending=True).round(6)
pvalue_percent

G2             0.000000
Intercept      0.005424
Faltas         0.157470
Reprobadas     2.671466
Sexo_num      16.819523
dtype: float64

In [110]:
# Definir quién será x, y
y = df[['G3']]
x = df[['Faltas', 'Reprobadas', 'G2', 'Sexo_num']]

Se dividieron los datos en conjuntos de entrenamiento (80%) y prueba (20%) utilizando train_test_split. El modelo de regresión lineal múltiple entrenado alcanzó un R2 de 0.83 en entrenamiento y 0.80 en prueba con una diferencia de solo 3.58%, lo que indica un buen equilibrio, sin overfitting. Las métricas de error muestran un RMSE de 2.36 puntos en el conjunto de prueba lo que representa un error aceptable considerando que las calificaciones estan entre los valores de 0 y 20. La ecuación final del modelo es: G3 = -1.21 + 0.05 * Faltas + -0.33 * Reprobadas + 1.07 * G2 + 0.21 * Sexo. Esto sugiere que G2 es la variable más influyente.

In [111]:
# Dividir datos en train y test
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((316, 4), (79, 4), (316, 1), (79, 1))

In [112]:
# Fórmulamos el modelo
modelRLM = LinearRegression()
# Entrenamos el modelo
modelRLM.fit(x_train, y_train)

LinearRegression()

In [113]:
# Validar si el modelo pronostica adecuadamente
y_pred_test = modelRLM.predict(x_test)
print(y_pred_test[0:5])
print(y_test.head())

[[13.95995181]
 [10.55870153]
 [ 9.91930646]
 [14.93423254]
 [16.00419488]]
     G3
329  14
318  10
317   9
65   15
59   16


In [114]:
R2_train = modelRLM.score(x_train, y_train)
print('R2 train = {:.2f}'.format(R2_train))

R2_test = modelRLM.score(x_test, y_test)
print('R2 test = {:.2f}'.format(R2_test))

print('Diferencia = {:.4f}%'.format(np.abs(R2_train-R2_test)*100))

R2 train = 0.83
R2 test = 0.80
Diferencia = 3.5837%


In [115]:
# Obtener el y_pred_train
y_pred_train = modelRLM.predict(x_train)

# Importar las librerías para el cálculo de error
from sklearn.metrics import mean_squared_error # MSE
from sklearn.metrics import mean_absolute_percentage_error # MAPE

# Calculamos los errores de train
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
mape_train = mean_absolute_percentage_error(y_train, y_pred_train)*100

print('MSE train: {:.2f}'.format(mse_train))
print('RMSE train: {:.2f}'.format(rmse_train))
print('MAPE train: {:.2f}%'.format(mape_train))

# Calculamos los errores de test
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)*100

print('\nMSE test: {:.2f}'.format(mse_test))
print('RMSE test: {:.2f}'.format(rmse_test))
print('MAPE test: {:.2f}%'.format(mape_test))

MSE train: 3.19
RMSE train: 1.79
MAPE train: 166352335914292512.00%

MSE test: 5.57
RMSE test: 2.36
MAPE test: 310604494973334080.00%


In [116]:
modelRLM.intercept_

array([-1.21088428])

In [117]:
modelRLM.coef_

array([[ 0.04784081, -0.3284302 ,  1.06996235,  0.2081894 ]])

In [118]:
print('B0 = {:.2f}'.format(modelRLM.intercept_[0]))
print('B1 = {:.2f}'.format(modelRLM.coef_[0][0]))
print('B2 = {:.2f}'.format(modelRLM.coef_[0][1]))
print('B3 = {:.2f}'.format(modelRLM.coef_[0][2]))
print('B4 = {:.2f}'.format(modelRLM.coef_[0][3]))
print('Ecuación de la recta')
print('G3 = {:.2f} + {:.2f} * Faltas + {:.2f} * Reprobadas + {:.2f} * G2 + {:.2f} * Sexo'.format(modelRLM.intercept_[0],modelRLM.coef_[0][0], modelRLM.coef_[0][1], modelRLM.coef_[0][2], modelRLM.coef_[0][3]))

B0 = -1.21
B1 = 0.05
B2 = -0.33
B3 = 1.07
B4 = 0.21
Ecuación de la recta
G3 = -1.21 + 0.05 * Faltas + -0.33 * Reprobadas + 1.07 * G2 + 0.21 * Sexo


In [119]:
x.head()

,Faltas,Reprobadas,G2,Sexo_num
0,6,0,6,0
1,4,0,5,0
2,10,3,8,0
3,2,0,14,0
4,4,0,10,0


In [120]:
modelRLM.predict([[7,2,8,0]])

/opt/anaconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[7.02683975]])

Despues de realizar la tarea, se pudo concluir que para predecir la calificación final (G3), lo más importante es fijarse en la calificación del período anterior (G2) y en las otras variables que se inluyeron en el modelo. Esto tiene sentido porque si a un estudiante le fue bien en el segundo período, es muy probable que le siga yendo bien al final. Este tipo de decisiones son sumamente importantes al momento de desarrollar modelos, el contexto de cada una de las variables afecta cómo se fórmula el modelo final.

La tarea me enseño sobre la importancia de la preparación de datos y la selección de características en el modelado predictivo. La transformación de variables categóricas y el análisis de correlaciones me ayudo a evitar problemas de multicolinealidad, mientras que la selección basada en significancia estadística pude producir un modelo interpretable. Probar modelos no lineales o ampliar el conjunto de datos con más características cualitativas podría ayudar para mejorar el rendimiento del modelo, utilizando otros como kNN, RandomForest, entre otros. En conclusión, el modelo desarrollado ofrece una base sólida para predecir el desempeño académico final, resaltando que el historial de calificaciones previas (G2) y el número de materias reprobadas son los factores más determinantes.

**He hecho esta actividad con integridad académica**

Referencias:

ICS (2014) Student Performance. Recuperado de : https://archive.ics.uci.edu/dataset/320/student+performance